<a href="https://colab.research.google.com/github/ChrisNiekler/NASDAQ-Dataset/blob/master/bert/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "transformers" muss installiert werden bevor man es nutzten kann.

In [3]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 8.5 MB/s 
     |████████████████████████████████| 636 kB 60.6 MB/s 
     |████████████████████████████████| 895 kB 64.0 MB/s 
     |████████████████████████████████| 3.3 MB 54.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Notwendige Libraries importieren

In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

# BERT sollte auf einer GPU laufen, da es auf einer CPU deutlich mehr Zeit benötigt.

In [2]:
# to check the GPU
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


## Lade das vortrainierte BERT Model und Tokenizer

In [3]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 3)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Laden der reinen Daten
BERT funktioniert ohne Tagging


In [11]:
#but the tweets.csv path
url = 'https://raw.githubusercontent.com/ChrisNiekler/NASDAQ-Dataset/master/Tweets.csv'
data = pd.read_csv(url)
data.head(5)

,Unnamed: 0,Tweets,Sentiment
0,0,Not surprised to see that #florists are mentio...,2.0
1,1,#Tesla Tint Complete! New vs Old @Tesla @elonm...,1.0
2,2,"Hai Phong #dog and #cat market, Thượng Thanh, ...",2.0
3,3,The mod list continues to grow... $tsla #tsla ...,1.0
4,4,5 Trades. 3 Wins. 2 Losses. P/L: +$891 1) $C...,1.0


# Prüfen ob sich Null-Werte in den Daten befinden

In [12]:
(data['Tweets'].isnull()).sum()

0

In [13]:
(data['Sentiment'].isnull()).sum()

0

# Null-Werte durch Neutrale Werte ersetzten
(Löschen wäre eine Alternative)

In [14]:
data = data.fillna(value=1.0)

In [15]:
(data['Tweets'].isnull()).sum()

0

In [16]:
(data['Sentiment'].isnull()).sum()

0

# Aufteilen der Daten in Feature und Label

In [17]:
x = data['Tweets']
y = data['Sentiment']

In [18]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2)

In [19]:
test_list = test_x.to_list()

In [20]:
test_list_2 = test_y.to_list()

In [21]:
# assign data of lists.
train_da = {'Tweet': train_x, 'Sentiment': train_y}
# Create DataFrame.
train_data = pd.DataFrame(train_da)
# Print the output.
print(train_data)

                                                   Tweet  Sentiment
7877   Let intuition light the way. "You have to trus...        1.0
761    💻 #Bitcoin $8633 vs 🚗 #TSLA $189           +8....        0.0
17452  Count how many things could go wrong trachea, ...        1.0
10745  It’s always a pleasure to hear great feedback ...        1.0
9521   I'm an ARMY without an army bomb, 0 albums, ne...        2.0
...                                                  ...        ...
58436  "Without music, life would be a mistake." 🖤 #P...        0.0
54814  If anyone has an #opinion, they should read th...        2.0
42644  PayPal Is Impressive, But It’s the Wrong Time ...        1.0
18298  Women’s World Cup final: A field guide to insu...        2.0
59658  Winter is Coming: Just the Blues or Seasonal A...        2.0

[49228 rows x 2 columns]


In [22]:
# assign data of lists.
test_data_dict = {'Tweet': test_x, 'Sentiment': test_y}
# Create DataFrame.
test_data = pd.DataFrame(test_data_dict)
# Print the output.
print(test_data)

                                                   Tweet  Sentiment
3658   IGEL Technology just won the 11th FASTEST GROW...        1.0
33740  Microsoft To Offer Paint As An Optional Featur...        0.0
28388  FYI @cyberabadpolice, the car involved is not ...        1.0
34698  It is hard to explain to people when they alre...        2.0
55531  You confuse an isolated incident, with a count...        1.0
...                                                  ...        ...
25783  Microsoft Updates Azure Server Licensing to Sq...        0.0
15354  $TSLA : $247... Getting into the big gap left ...        0.0
45076  A lot of us are only looking at the symptoms o...        1.0
14180  Just added an #NFLX game to the subs! That’s t...        1.0
17575  #TSLA #Tesla raises prices on standard Model 3...        0.0

[12307 rows x 2 columns]
